### Introduction

The project aims to build a model to predict drug efficacy of molecules.

A pre-processed HIV dataset with 3 classess (CA - Confirmed active, CM - Confirmed moderately active, CI - Confirmed inactive and benign) is available [here](http://moleculenet.ai/datasets-1). The raw data is available [here](https://wiki.nci.nih.gov/display/NCIDTPdata/AIDS+Antiviral+Screen+Data)
We perform stratified random splitting on the dataset: 80% of the images are in train set and 20% of the images are in test set.

#### Import required library

In [4]:
import os
import sys
sys.path.insert(0, os.getcwd())
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn import config_context
import sys
from warnings import catch_warnings, simplefilter
from logging import warning
from sklearn import feature_extraction
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, confusion_matrix
import pickle
import warnings
import argparse

In [34]:
path = os.getcwd()
dirname = os.path.dirname(path)
sys.path.append(dirname+'\\util')
import configs
from data import read_data, get_prediction_score
#from util import config


In [30]:
##########################
## FOLDER STURCTURE ######
##########################
WORK_DIRECTORY = 'C:/Users/liam.bui/Desktop/drug-efficacy/'
DATA_FILE = 'HIV.csv'

##########################
## EVALUATION METRICS ####
##########################
METRIC_ACCURACY = 'accuracy'
METRIC_F1_SCORE = 'f1-score'
METRIC_COHEN_KAPPA = 'Cohen kappa'
METRIC_CONFUSION_MATRIX = 'Confusion Matrix'

###############
## MODEL ######
###############
CLASSES = ['benign', 'malignant']
TEST_RATIO = 0.2
SEED = 0

In [3]:
def read_data(data_path, col_smiles='smiles', col_target='HIV_active'):
    """Split original data into train data and test data.
    :param data_path: str, path to the a CSV data file
    :param col_smiles: str, name of smiles column
    :param col_target: str, name of target column
    :param test_ratio: float, proportion of the original data for testset, must be from 0 to 1
    :param seed: int, randomization seed for reproducibility
    :return (X, y)
    """
    
    # read data
    df = pd.read_csv(data_path, sep=',')
    df_no_na = df[[col_smiles, col_target]].dropna()

    X = df_no_na[col_smiles]
    y = df_no_na[col_target].values
    
    return X, y

                
def get_prediction_score(y_label, y_predict):
    """Evaluate predictions using different evaluation metrics.
    :param y_label: list, contains true label
    :param y_predict: list, contains predicted label
    :return scores: dict, evaluation metrics on the prediction
    """
    scores = {}
    scores[config.METRIC_ACCURACY] = accuracy_score(y_label, y_predict)
    scores[config.METRIC_F1_SCORE] = f1_score(y_label, y_predict, labels=None, average='macro', sample_weight=None)
    scores[config.METRIC_COHEN_KAPPA] = cohen_kappa_score(y_label, y_predict)
    scores[config.METRIC_CONFUSION_MATRIX] = confusion_matrix(y_label, y_predict)
    
    return scores

In [5]:
def train_model(model, X_train, y_train, parameters, n_splits=3):
    '''Train model with Grid-Search corss validation to find the best hyparameter
    :param model: Scikit-Learn estimator
    :param X_train : train set features
    :param y_train: trainset label
    : param parameters: dict, key is hyper parametr name and value is a list of jyper parameter values
    return best_estimator: Scikit-learn estimator with the best hyper parameter
    :return best_score: best accuracy score
    :return best_score: best accuracy score
    :return best_param: dict, best hyper parameter
    '''

    splits = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0).split(X=X_train, y=y_train)
    clf = GridSearchCV(model, parameters, cv = splits, scoring=make_scorer(accuracy_score))
    with warnings.catch_warnings():
        warnings.catch_warnings('ignore')
        clf.fit(X_train, y_train)

    return clf.best_estimator_, clf.best_score_, clf.best_params_

In [6]:
def evaluate_model(model, X_train, y_trin, X_test,y_test):
    ''''
    Evaluae model on testset
    :param X_train: trainset features
    :param y_train: trainset label
    :param X_test: testset features
    :param ytest: testset label
    :param parameters dict, key is hyper arameter name and value is a list of hyper parameter values
    :return model: Scikit-lean estimator, fited on the whole trainset
    :return y_predict: prediction on test set
    :return scores: dict, evalutation metrics on test set
    '''

    # Refit the model on the whole train set
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        model.fit(X_train, y_train)
    
    # Evaluate on test set
    y_predict = model.predict(X_test)
    scores = None
    if y_test is not None:
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            scores = accuracy_score(y_test, y_predict)
    return model, y_predict, scores


In [7]:
def build_base_models(X_train, y_train):
    ''' Train and evaluate different base models. "Base" means the model is not a stacking model.
    :param X_train: trainset model
    :param y_train: trainset label
    :return fitted_models: list, contains fitted Scikit-learn estimators
    :return model_names: list, names of fitted Scikit-learn estimators
    :return model_scores: list, contains scores on test set for fitted Scikit-learn estimators;
                        Each score is a dict of evaluation metrics
    '''
    #######################
    # DEFINE BASE MODELS ##
    #######################
    
    models = []
    model_params = []
    model_names = []

    # Random forest model
    for n_estimators in [500, 1000, 2000]:
        for max_depth in [3, 5, 7]:
            models.append(RandomForestClassifier(max_features='sqrt', class_weight='balanced',random_state=0))
            model_params.append({'n_estimators':[n_estimators], 'max_depth':[max_depth]})
            model_names.append('Random Forest')
    
    # Gradient Boost
    for n_estimators in [500, 1000, 2000]:
        for max_depth in [3, 5, 7]:
            for learning_rate in [0.01, 0.1]:
                models.append(GradientBoostingClassifier(subsample=0.7, max_features='sqrt', random_state=0))
                model_params.append({'n_estimators':[n_estimators], 'max_depth':[max_depth], 'learning_rate':[learning_rate]})
                model_names.append('Gradient Boosting Machine')

    # Support Vector Machine
    for kernel in ['linear', 'rbf']:
        for C in [1.0, 10.0, 100.0, 1000.0]:
            models.append(SVC(probability=True, gamma ='auto', tol=0.001, cache_size=200, class_weight='balanced', random_state=0, decision_function_shape='ovr'))
            model_params.append({'kernel':[kernel],'C':[C]})
            model_names.append('Support Vector Machine')
            
    # Losgistic regression model
    for penalty in ['11', '12']:
        for C in [1.0, 10.0, 100.0, 1000.0]:
            models.append(linear_model.LogisticRegression(max_iter=500, solver='liblinear', multi_class='ovr',
                                                            class_weight='balanced', random_state=0))
            model_params.append({'penalty':[penalty], 'C':[C]})
            model_names.append('Logistic Regression')
    
    # K-nearest Neighbor
    for n_neighbors in [5, 10, 15]:
        for weights in ['uniform', 'distance']:
            models.append(KNeighborsClassifier())
            model_params.append({'n_neighbors':[n_neighbors], 'weights':[weights]})
            model_names.append('K Nearest Neighbor')

    ##################################
    # TRAIN AND EVALUATE BASE MODELS #
    ##################################
    fitted_models = []
    model_scores = []
    for i in range(len(models)):
        print("Evaluating model {} of {}: {}".format((i+1), len(models), model_names[i]))
        model = models[i]
        fitted_cv, _, _ = train_model(model=model, X_train= X_train, y_train=y_train, parameters= model_params[i])
        fitted_whole_set, _, score = evaluate_model(model=fitted_cv, X_train= X_train, y_train=y_train, X_test= X_test, y_test=y_test)
        fitted_models.append(fitted_models)
        model_scores.append(score)
        print(model_names[i], score)
    
    return fitted_models, model_names, model_scores

In [8]:
def build_stack_models(base_models,X_train, y_train):
    """Train and evaluate different stack models
    :param base models: list, contians fitted base models, which are Scikit-learn estimators
    :param X_train: trainset features
    :param y_train: trainset label
    :return stack_fitted_models: list, contains fitted Scikit-learn estimators
    :return stack_model_names: list, names of fitted Scikit-leanr estimators
    :return stack_model_scores: list, contains scores on test set for fitted Scikit-learn estimators.
                                        each score is a dict of evaluation metrics
    
    """
    ##############################
    ### PREPARE DATA FOR STACING #
    ##############################
    print('Preparing data for model stacking')
    # Get base modles's prediction for test set: simplyuse the tained models to predict on test set
    X_test_stack = np.zeros([X_test.shape[0], len[base_models]])
    for i in range(len(base_models)):
        model = base_models[i]
        X_test_stack[:,i] = model.predict(X_test)

    # Get base model's prediction for train set: use 3-fold split, train model on 2nd parts and paredict on 3rd part
    splits = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0).split(X=X_train,y=y_train)
    X_train_staack = np.zeros([X_train.shape[0], len(base_models)])
    for train_index, val_index in splits:
        # train and validation set
        X_tr, X_val = X_train[train_index], X_train[val_index]
        y_tr, _ = y_train[train_index], y_train[val_index]
        
        #Fit model
        for i in range(len(base_models)):
            model = base_models[i]
            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                model.fit(X_tr, y_tr)
            X_train_stack[val_index,i] = model.predict(X_val)

    # Add base model's predictions into the feature space
    X_train_stack = np.concatenate([X_train, X_train_stack], axis=-1)
    X_test_stack = np.concatenate([X_test, X_test_stack], axis=-1)

    #########################
    # DEFINE STACK MODELS ###
    #########################
    stack_models = []
    stack_model_names = []
    stack_model_params = []

    # logistic regression
    stack_models.append(linear_model.LogisticRegression(max_iter=500, solver='liblinear',multi_class='ovr', class_weight='balanced', random_state=0))
    stack_model_names.append('Stack Logistic Regression')
    stack_model_params.append({'penalty':['11','12'], 'C':[1.0,10.0, 100.0, 1000.0]})

    # Support Vecot Machine
    stack_models.append(SVC(probability=True, gamma='auto', tol=0.001, cache_size=200, class_weight= 'balanced', random_state=0, decision_function_shape='ovr'))
    stack_model_names.append('Stack Support Vector Machine')
    stack_model_params.append({'kernel':['linear','rbf'], 'C':[1.0, 10.0, 100.0, 1000.0]})

    # Random Forest
    stack_models.append(RandomForestClassifier(class_weight='balanced',random_state=0))
    stack_model_names.append('Stack Random Forest')
    stack_model_params.append({'n_estimators':[500, 1000, 2000], 'max_depth':[3, 5, 7]})

    # Gradient Boosting
    stack_models.append(GradientBoostingClassifier(subsample=0.7, max_features='sqrt', learning_rate=0.01, random_state=0))
    stack_model_names.append('Stack Gradient Boosting Machine')
    stack_model_params.append({'n_estimators':[500, 1000, 2000], 'max_depth':[3, 5, 7],'learning_rate':[0.01, 0.1]})

    # K-nearest Nieghbor
    stack_models.append(KNeighborsClassifier())
    stack_model_names.append('Stack KNN')
    stack_model_params.append({'n_neighbors':[5, 10, 15], 'weights':['uniform','distance']})


    ##########################
    # EVALUATE STACK MODELS #
    #########################
    stack_fitted_models = []
    stack_model_scores = []
    for i in range(len(stack_models)):
        print("Evaluating model {} of {}: {}".format((i+1), len(stack_models), stack_model_names[i]))
        model = stack_models[i]
        fitted_cv, _, _ = train_model(model, X_train = X_train_stack, y_train=y_train, parameters=stack_models[i])
        fitted_whole_set, _, score = evaluate_model(model=fitted_cv, X_train= X_train_stack, y_train=y_train, X_test=X_test_stack, y_test=y_test)
        stack_fitted_models.append(fitted_whole_set)
        stack_model_scores.append(score)
        print(stack_model_names[i], score)

    return stack_fitted_models, stack_model_names, stack_model_scores



In [37]:
configs

<module 'configs' from 'c:\\Users\\user\\OneDrive - City University of Hong Kong - Student\\CEVR\\Cheminformatics\\drug-efficacy-prediction-Zaw\\util\\configs.py'>

In [41]:
if __name__ == '__main__':
    data_path = os.path.join(configs.WORK_DIRECTORY,configs.DATA_FILE)
    n_splits = 3
    save_path = configs.WORK_DIRECTORY

    # parse parameters
    parser = argparse.ArgumentParser(description='Build base and stacking models')
    parser.add_argument('--data_path', help='A path to csv data file')
    parser.add_argument('--n_splits', type=int, help='Number of fold for Cross Validation')
    parser.add_argument('--save_path', help='A path to save fitted models')

    args = parser.parse_args()
    if args.data_path:
        data_path = args.data_path
    if args.n_splist:
        n_splits = args.n_splits
    if args.save_path:
        save_path = args.save_path

usage: ipykernel_launcher.py [-h] [--data_path DATA_PATH]
                             [--n_splits N_SPLITS] [--save_path SAVE_PATH]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9013 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"c1d4213c-73be-44ec-8f6d-316ddbec9af6" --shell=9007 --transport="tcp" --iopub=9014 --f=c:\Users\user\AppData\Roaming\jupyter\runtime\kernel-v2-21788SZ4N7sQmVLvf.json


SystemExit: 2